In [1]:
import requests
import json
import time
from typing import List, Dict, Optional
from xml.etree import ElementTree as ET

class VRRStationsAPI:
    """
    Klasse zum Abrufen aller Haltestellen über die VRR EFA API
    """

    def __init__(self):
        # EFA-basierte URL für VRR Stop Finder
        self.base_url = "https://efa.vrr.de/vrr/XSLT_STOPFINDER_REQUEST"

        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

    def test_api_connection(self):
        """
        Testet die EFA API-Verbindung
        """
        print("Teste EFA API-Verbindung...")
        test_params = {
            'outputFormat': 'XML',
            'language': 'de',
            'stateless': '1',
            'coordOutputFormat': 'WGS84[DD.DDDDD]',
            'locationServerActive': '1',
            'regionID_sf': '1',
            'SpEncId': '0',
            'odvSugMacro': 'true',
            'useHouseNumberList': 'true',
            'type_sf': 'any',
            'name_sf': 'Köln',
            'anyObjFilter_sf': '2'
        }

        try:
            response = requests.get(self.base_url, params=test_params, headers=self.headers, timeout=10)
            print(f"Status Code: {response.status_code}")
            print(f"Response URL: {response.url}")

            if response.status_code == 200:
                if 'itdOdv' in response.text or 'odvNameElem' in response.text:
                    print("✓ EFA API-Verbindung erfolgreich!")
                    return True
                else:
                    print("✗ Unerwartete Antwort von EFA API")
                    print("Response Text:", response.text[:300])
            else:
                print(f"✗ API-Fehler: {response.status_code}")
                print("Response Text:", response.text[:200])

        except requests.RequestException as e:
            print(f"✗ Verbindungsfehler: {e}")

        return False

    def _fetch_and_parse_stations(self, params: Dict, max_results: Optional[int] = None) -> List[Dict]:
        """
        Interne Methode zum Abrufen und Parsen von Haltestellendaten von der EFA API.
        """
        stations = []
        try:
            response = requests.get(self.base_url, params=params, headers=self.headers)
            response.raise_for_status()

            root = ET.fromstring(response.text)

            for elem in root.iter('odvNameElem'):
                try:
                    name = elem.text or ""
                    station_id = elem.get('id', '')

                    coord_elem = elem.find('.//coord')
                    coords = []
                    if coord_elem is not None:
                        x = coord_elem.get('x', '')
                        y = coord_elem.get('y', '')
                        if x and y:
                            coords = [float(x), float(y)]

                    station_info = {
                        'name': name,
                        'id': station_id,
                        'coord': coords,
                        'type': 'stop'
                    }
                    stations.append(station_info)

                    if max_results is not None and len(stations) >= max_results:
                        break

                except Exception as e:
                    # Log or handle specific parsing errors if needed
                    continue

            return stations

        except requests.RequestException as e:
            print(f"Fehler beim API-Aufruf: {e}")
            return []
        except ET.ParseError as e:
            print(f"Fehler beim XML-Parsing: {e}")
            return []


    def get_stations_by_name_pattern(self, pattern: str = "", max_results: int = 1000) -> List[Dict]:
        """
        Holt Haltestellen basierend auf einem Namenspattern über EFA

        Args:
            pattern: Suchpattern (leer für alle)
            max_results: Maximum Anzahl Ergebnisse

        Returns:
            Liste mit Haltestellen-Dictionaries
        """
        params = {
            'outputFormat': 'XML',
            'language': 'de',
            'stateless': '1',
            'coordOutputFormat': 'WGS84[DD.DDDDD]',
            'locationServerActive': '1',
            'regionID_sf': '1',
            'SpEncId': '0',
            'odvSugMacro': 'true',
            'useHouseNumberList': 'true',
            'type_sf': 'stop',
            'name_sf': pattern or '*',
            'anyObjFilter_sf': '2'
        }
        return self._fetch_and_parse_stations(params, max_results)


    def get_comprehensive_station_list(self) -> List[Dict]:
        """
        Versucht eine umfassende Liste aller Haltestellen zu erstellen
        durch verschiedene Suchmethoden
        """
        print("Sammle Haltestellen...")
        all_stations = {}

        # Suche nach häufigen Präfixen
        common_prefixes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'L', 'M', 'N', 'O', 'S', 'U', 'W']

        for prefix in common_prefixes:
            print(f"Suche nach Präfix '{prefix}'...")
            stations = self.get_stations_by_name_pattern(prefix, 1000)
            for station in stations:
                all_stations[station['id']] = station
            time.sleep(0.5)  # Höfliche Pause zwischen Anfragen

        print(f"Insgesamt {len(all_stations)} eindeutige Haltestellen gefunden.")
        return list(all_stations.values())

    def save_stations_to_file(self, stations: List[Dict], filename: str = "vrr_haltestellen.json"):
        """
        Speichert die Haltestellen in einer JSON-Datei
        """
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(stations, f, ensure_ascii=False, indent=2)
            print(f"Haltestellen erfolgreich in '{filename}' gespeichert.")
        except Exception as e:
            print(f"Fehler beim Speichern: {e}")

    def print_station_summary(self, stations: List[Dict]):
        """
        Gibt eine Zusammenfassung der gefundenen Haltestellen aus
        """
        if not stations:
            print("Keine Haltestellen gefunden.")
            return

        print(f"\n=== ZUSAMMENFASSUNG ===")
        print(f"Anzahl Haltestellen: {len(stations)}")
        print(f"\nErste 10 Haltestellen:")
        for i, station in enumerate(stations[:10]):
            coord_info = ""
            if station.get('coord') and len(station['coord']) >= 2:
                coord_info = f" (Lat: {station['coord'][1]:.6f}, Lon: {station['coord'][0]:.6f}) "
            print(f"{i+1:2d}. {station['name']} [ID: {station['id']}] {coord_info}")

        if len(stations) > 10:
            print(f"... und {len(stations) - 10} weitere Haltestellen")

# Hauptprogramm
if __name__ == "__main__":
    # API-Instanz erstellen
    vrr_api = VRRStationsAPI()

    # Zuerst API-Verbindung testen
    if not vrr_api.test_api_connection():
        print("\nAPI-Verbindung fehlgeschlagen. Bitte prüfen Sie:")
        print("1. Internetverbindung")
        print("2. API-URL (möglicherweise geändert)")
        print("3. Eventuell sind zusätzliche Parameter erforderlich")
        exit(1)

    print("\nVRR Haltestellen-Abfrage gestartet...")
    print("=" * 50)

    # Alle Haltestellen abrufen
    all_stations = vrr_api.get_comprehensive_station_list()

    # Ergebnisse anzeigen
    vrr_api.print_station_summary(all_stations)

    # In Datei speichern
    vrr_api.save_stations_to_file(all_stations)

    print(f"\nFertig! Insgesamt {len(all_stations)} Haltestellen gefunden.")
    print("Die vollständige Liste wurde in 'vrr_haltestellen.json' gespeichert.")

Teste EFA API-Verbindung...
Status Code: 200
Response URL: https://efa.vrr.de/vrr/XSLT_STOPFINDER_REQUEST?outputFormat=XML&language=de&stateless=1&coordOutputFormat=WGS84%5BDD.DDDDD%5D&locationServerActive=1&regionID_sf=1&SpEncId=0&odvSugMacro=true&useHouseNumberList=true&type_sf=any&name_sf=K%C3%B6ln&anyObjFilter_sf=2
✓ EFA API-Verbindung erfolgreich!

VRR Haltestellen-Abfrage gestartet...
Sammle Haltestellen...
Suche nach Präfix 'A'...
Suche nach Präfix 'B'...
Suche nach Präfix 'C'...
Suche nach Präfix 'D'...
Suche nach Präfix 'E'...
Suche nach Präfix 'F'...
Suche nach Präfix 'G'...
Suche nach Präfix 'H'...
Suche nach Präfix 'K'...
Suche nach Präfix 'L'...
Suche nach Präfix 'M'...
Suche nach Präfix 'N'...
Suche nach Präfix 'O'...
Suche nach Präfix 'S'...
Suche nach Präfix 'U'...
Suche nach Präfix 'W'...
Insgesamt 6266 eindeutige Haltestellen gefunden.

=== ZUSAMMENFASSUNG ===
Anzahl Haltestellen: 6266

Erste 10 Haltestellen:
 1. Ahlen (Westf), Isendorf, Abzw. A. MÃ¼nsterstr. [ID: 240